In [29]:
from nets import Actor
import tensorflow as tf
import numpy as np

In [20]:
actor = Actor(nature = "target")

In [21]:
actor.lstm.stateful

False

In [26]:
for k in range(40,43,1):
    inps = tf.random.uniform((k,1,1))
    actor(inps)

In [27]:
# @tf.function
def process_sequence_of_experiences_tf(actor_target, experiences):

    unstacked_exp = tf.unstack(tf.convert_to_tensor(experiences), axis=1)
    to_stack = []
    for index in range(2*actor_target.dolinar_layers-1): # I consider from first outcome to last one (but guess)
        if (index==0):
            to_stack.append(unstacked_exp[index])
        if (index%2 == 1):
            to_stack.append(unstacked_exp[index])

            to_stack.append(tf.squeeze(actor_target(tf.reshape(unstacked_exp[index],(experiences.shape[0],1,1)))))
    for index in range(2*actor_target.dolinar_layers-1, 2*actor_target.dolinar_layers+2):
        to_stack.append(unstacked_exp[index])

    return tf.stack(to_stack, axis=1)
    #

In [33]:
experiences = np.load("tutorials_functions/expe_2L.npy").astype(np.float32)

In [34]:
process_sequence_of_experiences_tf(actor, experiences)

<tf.Tensor: shape=(100, 6), dtype=float32, numpy=
array([[ 0.6910216 ,  0.        ,  0.05255208,  0.        , -1.        ,
         1.        ],
       [ 0.98145694,  0.        ,  0.05255208,  0.        , -1.        ,
         1.        ],
       [ 0.695889  ,  0.        ,  0.05255208,  0.        ,  1.        ,
         0.        ],
       [ 0.654807  ,  0.        ,  0.05255208,  1.        , -1.        ,
         0.        ],
       [ 0.07486903,  0.        ,  0.05255208,  1.        ,  1.        ,
         1.        ],
       [ 0.45389345,  1.        ,  0.04730617,  0.        ,  1.        ,
         1.        ],
       [ 0.77413857,  1.        ,  0.04730617,  0.        ,  1.        ,
         1.        ],
       [ 0.8683654 ,  0.        ,  0.05255208,  0.        ,  1.        ,
         0.        ],
       [ 0.74878466,  0.        ,  0.05255208,  1.        , -1.        ,
         0.        ],
       [ 0.5304443 ,  0.        ,  0.05255208,  1.        , -1.        ,
         0.        ],


In [36]:
actor_target = Actor(nature="target")
actor_target.lstm.stateful=True
unstacked_exp = tf.unstack(tf.convert_to_tensor(experiences), axis=1)
to_stack = []
for index in range(2*actor_target.dolinar_layers-1): # I consider from first outcome to last one (but guess)
    if (index==0):
        to_stack.append(unstacked_exp[index])
    if (index%2 == 1):
        to_stack.append(unstacked_exp[index])

        to_stack.append(tf.squeeze(actor_target(tf.reshape(unstacked_exp[index],(experiences.shape[0],1,1)))))
for index in range(2*actor_target.dolinar_layers-1, 2*actor_target.dolinar_layers+2):
    to_stack.append(unstacked_exp[index])

In [38]:
actor_target.reset_states()

In [39]:
experiences=experiences[:5]
actor_target.lstm.stateful=True
unstacked_exp = tf.unstack(tf.convert_to_tensor(experiences), axis=1)
to_stack = []
for index in range(2*actor_target.dolinar_layers-1): # I consider from first outcome to last one (but guess)
    if (index==0):
        to_stack.append(unstacked_exp[index])
    if (index%2 == 1):
        to_stack.append(unstacked_exp[index])

        to_stack.append(tf.squeeze(actor_target(tf.reshape(unstacked_exp[index],(experiences.shape[0],1,1)))))
for index in range(2*actor_target.dolinar_layers-1, 2*actor_target.dolinar_layers+2):
    to_stack.append(unstacked_exp[index])

InvalidArgumentError: Incompatible shapes: [5,2000] vs. [100,2000] [Op:AddV2]

In [40]:
experiences

array([[ 0.6910216 ,  0.        ,  0.8369508 ,  0.        , -1.        ,
         1.        ],
       [ 0.98145694,  0.        ,  0.04985608,  0.        , -1.        ,
         1.        ],
       [ 0.695889  ,  0.        ,  0.30091032,  0.        ,  1.        ,
         0.        ],
       [ 0.654807  ,  0.        ,  0.72952473,  1.        , -1.        ,
         0.        ],
       [ 0.07486903,  0.        ,  0.39324445,  1.        ,  1.        ,
         1.        ]], dtype=float32)

In [ ]:
def actor_grad_tf(actor, dq_da, experiences, optimizer_actor):
    with tf.GradientTape() as tape:
        unstacked_exp = tf.unstack(tf.convert_to_tensor(experiences), axis=1)
        states_to_act=[tf.ones((experiences.shape[0],1,1))*actor.pad_value]

        to_stack = []
        actions_wathed_index = []
        for index in range(1,2*actor.dolinar_layers-2,2):
            states_to_act.append(tf.reshape(unstacked_exp[index],(experiences.shape[0],1,1)))
        inps_actor = tf.concat(states_to_act, axis=1)
        actor.lstm.stateful=False
        actor_thinks = actor(inps_actor)
        actor.lstm.stateful=True
        da_dtheta = tape.gradient(actor_thinks, actor.trainable_variables, output_gradients=-dq_da)
        optimizer_actor.apply_gradients(zip(da_dtheta, actor.trainable_variables))
    return
